# Experimento de analise geral.

## TL;DR

nesse notebook estudamos como toda nossa exploração se combina e gera uma analise que abrange um escopo maior para 

## Objetivo

estudar como carda tipo de análise desde como cada tipo de pré processamento afeta os dados, até a analise das métricas que cada um dos aplicação dos métodos de clusterização

## Dados

CSV's





# Experimento 01

## Import das Bibliotecas


In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Bloco 1 - K-Means tradicional com distância Euclidiana
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.cm as cm
# Bloco 2 - K-Means com distância DTW (tslearn)
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
# Bloco 3 - Hierarchical Clustering com DTW
from scipy.cluster.hierarchy import linkage, fcluster
from tslearn.metrics import cdist_dtw
# Bloco 4 - DBA-KMeans (K-Means com DTW Barycenter Averaging)
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
import seaborn as sns

from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

In [80]:
df_original = pd.read_csv('../../data/rafaelDB.csv')

In [81]:
df_metricas = pd.DataFrame(columns=[
    'Metodo',
    'Ajuste'
    'n_clusters',
    'Silhouette Score',
    'Calinski-Harabasz Score',
    'Davies-Bouldin Score'
])

dic_ajustes = ['media', 'mediana', 'zero', 'trim']
dic_metodos = ['kMeans', 'kMeansDTW', 'Aglomerative', 'DBAKmeansDTW0']


## Funções auxiliares

In [82]:
def ajuste_dados(num, dfOriginal):
    df = dfOriginal.copy()
    
    match num:
        case 0:
            # ajuste 0 -> media
            df.iloc[1:] = df.iloc[1:].apply(lambda col: col.fillna(col.mean()), axis=0)
        case 1:
            # ajuste 2 -> mediana
            df.iloc[1:] = df.iloc[1:].apply(lambda col: col.fillna(col.median()), axis=0)
        case 2:
            #ajuste 3 -> zeros
            df.iloc[1:] = df.iloc[1:].fillna(0)
        case 3:
            #ajuste 4 -> trim
            timestamp = df.iloc[:, 0]
            dados = df.iloc[:, 1:]

            # Conta valores não nulos (validos) por coluna (sensor)
            valid_lengths = dados.notna().sum(axis=0)

            # Identifica o menor tamanho válido
            min_length = valid_lengths.min()

            # Corta todas as colunas para o menor tamanho
            dados_cortados = dados.iloc[:min_length, :]

            # Corta o timestamp também para manter alinhamento
            timestamp_cortado = timestamp.iloc[:min_length]

            # Reconstroi DataFrame final
            df = pd.concat([timestamp_cortado.reset_index(drop=True), dados_cortados.reset_index(drop=True)], axis=1)

    return df.T

In [83]:
def separadataset(df):
    # timestamp (eixo X)
    timestamp = df.iloc[0].values

    # séries temporais (linhas 1 a n)
    series = df.iloc[1:].values
    return timestamp, series


## Função Principal para o experimento

In [ ]:
#mudando os ajustes dos dados (normalização etc)
for indice_ajuste in range(4):
    #ajustando os dados no dfAjustado
    dfAjustado = ajuste_dados(indice_ajuste,df_original)

    print("indice ajuste: ", indice_ajuste)

    #mudando os metodos
    for indice_metodo in range(4):
        print("indice do metodo: ", indice_metodo)
    
        #mudando a quantidade de clusters
        for qnt_cluster in range(2,9):
            print("Qnt de clusters: ", qnt_cluster)
            
            #copia o dfAjustado já transposto
            df = dfAjustado.copy()

            #pega o x e y
            timestamp,series = separadataset(df)

            #normaliza o X do DF
            #X_scaled = TimeSeriesScalerMeanVariance().fit_transform(series)
            X_scaled = StandardScaler().fit_transform(series)

            # ----------- isso tudo é genérico, agora vai ser o switch case
            match indice_metodo:

                case 0: # K MEANS
                    
                    kmeans = KMeans(n_clusters=qnt_cluster, random_state=42)
                    clusters = kmeans.fit_predict(X_scaled)
                    #calcular inertia

                case 1: # K MEANS DTW
                    
                    km_dtw = TimeSeriesKMeans(n_clusters=qnt_cluster, metric="dtw", random_state=42)
                    clusters = km_dtw.fit_predict(X_scaled)
                    # calcular inetia

                case 2: # AGLOMERATIVE DTW

                    # Calcular a matriz de distância DTW
                    distance_matrix = cdist_dtw(X_scaled)
                    # Aplicar Hierarchical Clustering com DTW
                    Z = linkage(distance_matrix, method='average')
                    clusters = fcluster(Z, t=qnt_cluster, criterion='maxclust') #<- esse fclusters tá atrapalhando o plot
                    #calcular inertia

                case 3: # DBA K MEANS DTW

                    km_dba = TimeSeriesKMeans(n_clusters=qnt_cluster, metric="dtw", random_state=42)
                    clusters = km_dba.fit_predict(X_scaled)
                    #calcular inertia
            
                
            # Calcular métricas de clusterização internas (sem y_true)
            silhouette = silhouette_score(X_scaled, clusters)
            calinski = calinski_harabasz_score(X_scaled, clusters)
            davies = davies_bouldin_score(X_scaled, clusters)
            # Crie um dicionário com os dados da nova linha

            nova_linha = {
                'Metodo': dic_metodos[indice_metodo],
                'Ajuste': dic_ajustes[indice_ajuste],
                'Quantidade_Cluster': qnt_cluster,
                'Silhouette': silhouette,
                'Calinski_Harabasz': calinski,
                'Davies_Bouldin': davies
                #inertia 
            }
            #append inertia 

            # Crie um DataFrame a partir do dicionário da nova linha
            df_nova_linha = pd.DataFrame([nova_linha])
            # Concatene o DataFrame existente com o DataFrame da nova linha
            df_metricas = pd.concat([df_metricas, df_nova_linha], ignore_index=True)

            #df_metricas = df_metricas.append(dic_metodos[indice_metodo],dic_ajustes[indice_ajuste], qnt_cluster, silhouette, calinski, davies)
            
    

In [ ]:
df_metricas

,Metodo,Ajusten_clusters,Silhouette Score,Calinski-Harabasz Score,Davies-Bouldin Score,Ajuste,Quantidade_Cluster,Silhouette,Calinski_Harabasz,Davies_Bouldin
0,kMeans,NaN,NaN,NaN,NaN,media,2.0,0.611127,95.784832,0.555415
1,kMeans,NaN,NaN,NaN,NaN,media,3.0,0.643906,159.950503,0.469171
2,kMeans,NaN,NaN,NaN,NaN,media,4.0,0.685619,219.503266,0.421719
3,kMeans,NaN,NaN,NaN,NaN,media,5.0,0.577676,167.638380,0.452882
4,kMeans,NaN,NaN,NaN,NaN,media,6.0,0.569179,197.288238,0.442196
...,...,...,...,...,...,...,...,...,...,...
132,DBAKmeansDTW0,NaN,NaN,NaN,NaN,trim,4.0,0.604044,109.328089,0.543373
133,DBAKmeansDTW0,NaN,NaN,NaN,NaN,trim,5.0,0.655691,140.065646,0.455797
134,DBAKmeansDTW0,NaN,NaN,NaN,NaN,trim,6.0,0.608917,116.610193,0.483689
135,DBAKmeansDTW0,NaN,NaN,NaN,NaN,trim,7.0,0.546835,98.940498,0.705245


In [93]:
df_metricas.to_csv('Metricas.csv', index=False)